## Data Preparation

In [ ]:
import pandas as pd
import json
import seaborn as sns

from dotenv import load_dotenv
from pathlib import Path
from studienarbeit.utils.cleaning import Cleaning
from studienarbeit.utils.split_text import SplitText
from studienarbeit.utils.load import EDataTypes
from tqdm import tqdm

In [ ]:
load_dotenv()
sns.set(style="white", palette="muted", rc={"figure.figsize": (20, 8)})
tqdm.pandas()

In [ ]:
data_dir = Path("../../data/party_programs")
data_type = EDataTypes.PARTY_PROGRAMS

In [ ]:
with open("../../data/party_colors.json", "r", encoding="utf-8") as f:
    party_palette = json.load(f)
cleaning = Cleaning()
split_text = SplitText()

In [ ]:
SENTENCE_LEVEL = True
MIN_WORD_COUNT = 5

In [ ]:
df_prep = pd.read_parquet(data_dir / "party_programs_before_cleaning.parquet")

In [ ]:
df_prep.duplicated(subset=["text_orig"]).sum()

In [ ]:
df_prep = df_prep.drop_duplicates(subset=["text_orig"]).reset_index(drop=True)

In [ ]:
df_prep.duplicated(subset=["text_orig"]).sum()

In [ ]:
df_prep["clean_text"] = (
    df_prep["text_orig"]
    .progress_apply(lambda x: cleaning.clean_text(x, keep_punctuation=True, keep_upper=True))
    .astype("string[pyarrow]")
)

df_prep.drop(columns=["text_orig", "election_type", "election"], inplace=True)
df_prep

In [ ]:
if SENTENCE_LEVEL:
    df_prep = split_text.split_dataframe_texts(df_prep, "clean_text", None, SENTENCE_LEVEL)
df_prep

In [ ]:
df_prep["word_count"] = df_prep["clean_text"].progress_apply(lambda x: len(x.split()))
df_prep = df_prep[df_prep["word_count"] > MIN_WORD_COUNT].reset_index(drop=True).drop(columns=["word_count"])
df_prep

In [ ]:
df_prep["tokenized_text"] = (
    df_prep["clean_text"]
    .progress_apply(lambda x: cleaning.filter_text(cleaning.lemma_text(x)))
    .astype("string[pyarrow]")
)
df_prep

In [ ]:
df_final = df_prep.copy().reset_index(drop=True)

In [ ]:
with open("../../data/party_encoding.json", "r", encoding="utf-8") as f:
    party_encoding = json.load(f)

df_final["party"] = df_final["party"].map(party_encoding)
df_final

In [ ]:
if SENTENCE_LEVEL:
    df_final.to_parquet(data_dir / "party_programs_sentence.parquet", index=False)
else:
    df_final.to_parquet(data_dir / "party_programs.parquet", index=False)